In [342]:
pdf_folder_path = "/Users/badekale/Documents/Hamoyeew/Subfolders"

In [343]:
import os
subfolders = sorted([folder for folder in os.listdir(pdf_folder_path) if os.path.isdir(os.path.join(pdf_folder_path, folder))])
print(f"Total files in the folder: {len(subfolders)}")

Total files in the folder: 52


In [344]:
import os
from langchain_nomic import NomicEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_nomic import NomicEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.chains import RetrievalQA
from IPython.display import Markdown
import re

In [345]:
subfolders = [folder for folder in os.listdir(pdf_folder_path) if os.path.isdir(os.path.join(pdf_folder_path, folder))]

# Function to sort subfolders separately by type (COP, CMP, CMA) in descending order
def custom_sort_key(folder_name):
    # Match COP, CMP, CMA with their numbers
    match = re.match(r"(COP|CMP|CMA)(\d+)", folder_name)
    if match:
        conference_type = match.group(1)
        number = int(match.group(2))
        if conference_type == "COP":
            return (0, -number)  # Sort COP first in descending order
        elif conference_type == "CMP":
            return (1, -number)  # CMP comes after COP, in descending order
        elif conference_type == "CMA":
            return (2, -number)  # CMA comes last, in descending order
    return (3, folder_name)  # Default case if no match

# Sort the subfolders by type (COP, CMP, CMA)
cop_subfolders = [folder for folder in subfolders if "COP" in folder]
cmp_subfolders = [folder for folder in subfolders if "CMP" in folder]
cma_subfolders = [folder for folder in subfolders if "CMA" in folder]

# Sort each list in descending order
cop_subfolders.sort(key=custom_sort_key)
cmp_subfolders.sort(key=custom_sort_key)
cma_subfolders.sort(key=custom_sort_key)

# Create a list to store the final ordered subfolders by alternating COP, CMP, and CMA
final_order = []

# Find the maximum number of COP, CMP, and CMA folders to ensure alternating
max_length = max(len(cop_subfolders), len(cmp_subfolders), len(cma_subfolders))

# Alternate between COP, CMP, and CMA
for i in range(max_length):
    if i < len(cop_subfolders):
        final_order.append(cop_subfolders[i])
    if i < len(cmp_subfolders):
        final_order.append(cmp_subfolders[i])
    if i < len(cma_subfolders):
        final_order.append(cma_subfolders[i])

# Create a list to store the documents
documents = []

# Initialize a counter for the number of PDFs ingested
pdf_count = 0

# Function to load and process PDFs from each subfolder
def load_pdfs_from_subfolders(pdf_folder_path, final_order):
    global pdf_count

    for subfolder in final_order:
        # Get the path for the current subfolder
        subfolder_path = os.path.join(pdf_folder_path, subfolder)
        
        # List all PDFs in the subfolder and sort them
        files_in_subfolder = sorted([file for file in os.listdir(subfolder_path) if file.endswith('.pdf')])

        # Print the subfolder name being processed
        print(f"Processing folder: {subfolder}")

        # Load each PDF in the subfolder
        for file_name in files_in_subfolder:
            pdf_path = os.path.join(subfolder_path, file_name)
            loader = PyPDFLoader(pdf_path)
            doc = loader.load()
            
            # Add the loaded document to the list
            documents.extend(doc)
            
            # Increment the PDF counter
            global pdf_count
            pdf_count += 1
            print(f'Loaded and processed: {file_name}')

# Load PDFs from the sorted subfolders
load_pdfs_from_subfolders(pdf_folder_path, final_order)

Processing folder: COP28
Loaded and processed: cp2023_11a01E.pdf
Loaded and processed: cp2023_11a02E.pdf
Processing folder: CMP18
Loaded and processed: cmp2023_09a01E.pdf
Processing folder: CMA5
Loaded and processed: cma2023_16a01E.pdf
Loaded and processed: cma2023_16a02E.pdf
Loaded and processed: cma2023_16a03E.pdf
Processing folder: COP27
Loaded and processed: cp2022_10a01_E.pdf
Loaded and processed: cp2022_10a02E.pdf
Loaded and processed: cp2022_10a03E.pdf
Processing folder: CMP17
Loaded and processed: cmp2022_09a01E.pdf
Processing folder: CMA4
Loaded and processed: cma2022_10_a01E.pdf
Loaded and processed: cma2022_10a03E.pdf
Loaded and processed: cma2023_10a02E.pdf
Processing folder: COP26
Loaded and processed: cp2021_12_add1E.pdf
Loaded and processed: cp2021_12a02E.pdf
Processing folder: CMP16
Loaded and processed: cmp2021_08_add1E.pdf
Processing folder: CMA3
Loaded and processed: CMA2021_10_Add3_E.pdf
Loaded and processed: CMA2021_L10a2E.pdf
Loaded and processed: cma2021_10a01E.p

In [347]:
documents[1].page_content

'FCCC/CP/2023/11/Add.1 \n2  \n   Decision 1/CP.28 \n  Operationalization of the new funding arrangements, \nincluding a fund, for responding to loss and damage referred \nto in paragraphs 2–3 of decisions 2/CP.27 and 2/CMA.4 \n  The Conference of the Parties and the Conference of the Parties serving as the \nmeeting of the Parties to the Paris Agreement, \n  Recalling decisions 2/CP.27 and 2/CMA.4, \n Also recalling decisions 2/CP.27, paragraph 2, and 2/CMA.4, paragraph 2, by which \nnew funding arrangements were establi shed for assisting developing countries that are \nparticularly vulnerable to the adverse effects of climate change in responding to loss and \ndamage, including with a focus on addressing loss and damage , by providing and assisting \nin mobilizing new and addit ional resources, and which specify that these new arrangements \ncomplement and include sources, funds, processes and initiatives under and outside the \nConvention and the Paris Agreement, \n Further recallin

In [348]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

In [378]:
print(f'Total number of document chunks: {len(chunks)}')

Total number of document chunks: 12339


In [379]:
chunks[12338].page_content

'guidance that the Conference of the Parties may provide in the future, the interim secretariat\nwas requested to continue to facilitate the provision of technical and financial support to\nParties, in cooperation with its partners, and to periodically report back to the Conference of\nthe Parties on progress achieved, so that further guidance could be provided."\n-----'

In [351]:
from langchain_ollama import OllamaEmbeddings

In [352]:
embeddings_model = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [353]:
from langchain_chroma import Chroma


In [402]:
vectorstore = Chroma.from_documents(
    documents=documents,
    collection_name="rag-chroma",
    embedding=embeddings_model,
    persist_directory='/Users/badekale/Documents/Hamoyeew/chromanir_index3'
)


In [422]:
question = "Report of the Conference of the Parties serving as the meeting of the Parties to the Paris Agreement on the first part of its first session, held in Marrakech"
docs = vectorstore.similarity_search(question, k=10)
len(docs)

10

In [404]:
docs[4]

Document(metadata={'page': 23, 'source': '/Users/badekale/Documents/Hamoyeew/Subfolders/CMP11/08a02-2.pdf'}, page_content='FCCC/KP/CMP/2015/8/Add.2 \n24  \nResolution 1/CMP.11 \n  Expression of gratitude to the Government of the French \nRepublic and the people of the city of Paris \nDraft resolution submitted by Morocco \nThe Conference of the Parties and the Conference of the Parties serving as the \nmeeting of the Parties to the Kyoto Protocol, \nHaving met in Paris from 30 November to 11 December 2015 at the invitation of the \nGovernment of the French Republic, \n1. Express their profound gratitude  to the Government of the Frenc h Republic for \nhaving made it possible for the twenty -first session of the Conference of the Parties and the \neleventh session of the Conference of the Parties serving as the meeting of the Parties to the \nKyoto Protocol to be held in Paris; \n2. Request the Government of the French Republic to convey to the city and people of \nParis the gratitude o

In [383]:
question = "climate policy themes"
doc = vectorstore.similarity_search(question, k=10)
len(doc)

10

In [384]:
doc[9]

Document(metadata={'page': 21, 'source': '/Users/badekale/Documents/Hamoyeew/Subfolders/COP18/08a02.pdf'}, page_content='FCCC/CP/2012/8/Add.2 \n22  \nsubregional, regional and international levels, as appropriate.  Technical skills and \nknowledge provide an opportunity to adequately address and res pond to climate change \nissues. \n C. Public awareness  \n18. Cooperate in, promote, fac ilitate, develop and implement public awareness \nprogrammes on climate change and its effects at the national and, as appropriate, \nsubregional, regional and international levels by, inter alia, encouraging contributions and \npersonal action in addressing cli mate change, supporting climate -friendly policies and \nfostering behavioural changes, including by using popular media, noting the important role \nthat social media platforms and strategies can play in this context.  \n D. Public access to information \n19. Facilitate public access to data and information, by providing the information on \nc

In [420]:
from langchain_ollama import OllamaLLM

ollama = OllamaLLM(
    base_url="http://localhost:11434",  
    model="llama3.2:3b",  
    temperature=0,  
    #num_predict=1000  
)


In [ ]:
sample_response = ollama.invoke("What is the impact of climate change on coastal cities?")
print(sample_response)


The impact of climate change on coastal cities can be severe and far-reaching. Some of the most significant effects include:

1. Sea-level rise: As global temperatures rise, the oceans are expanding, causing sea levels to rise. This can lead to flooding, erosion, and saltwater intrusion into freshwater sources.
2. Increased storm intensity: Climate change is projected to increase the frequency and intensity of storms, which can cause catastrophic damage to coastal cities and infrastructure.
3. Coastal erosion: Rising sea levels and increased storm activity can lead to coastal erosion, causing beaches to disappear and coastal defenses to be breached.
4. Saltwater intrusion: As sea levels rise, saltwater can intrude into freshwater sources, making them undrinkable and affecting agriculture and industry.
5. Loss of infrastructure: Climate change can cause flooding and damage to coastal infrastructure, including roads, bridges, and buildings.
6. Displacement of communities: Rising sea leve

In [421]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
rag_chain = RetrievalQA.from_chain_type(
    llm=ollama,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


In [408]:
response = rag_chain.invoke("All Climate policies themes mentioned in the documents")
print(response)

{'query': 'All Climate policies themes mentioned in the documents', 'result': 'Based on the provided documents, the climate policy themes mentioned are:\n\n1. Education\n2. Training\n3. Public awareness\n4. Public access to information\n5. Public participation\n6. International cooperation\n7. National circumstances and capacities\n8. Sustainable development\n9. Adaptation (specifically, the National Adaptation Plans of Action (NAPAs))\n\nThese themes are mentioned in various sections of the documents, including the FCCC/CP work programmes, the New Delhi work programme, and the NAPA document structure.', 'source_documents': [Document(metadata={'page': 44, 'source': '/Users/badekale/Documents/Hamoyeew/Subfolders/COP2/15a01.pdf'}, page_content='FCCC/CP/1996/15/Add.1\nEnglish\nPage 45\n(c) Any other information that the Party considers relevant to the achievement of\nthe objective of the Convention and suitable for inclusion in its communication, including, if\nfeasible, material relevant

In [355]:
response.keys()

dict_keys(['query', 'result'])

In [409]:
response["result"]

'Based on the provided documents, the climate policy themes mentioned are:\n\n1. Education\n2. Training\n3. Public awareness\n4. Public access to information\n5. Public participation\n6. International cooperation\n7. National circumstances and capacities\n8. Sustainable development\n9. Adaptation (specifically, the National Adaptation Plans of Action (NAPAs))\n\nThese themes are mentioned in various sections of the documents, including the FCCC/CP work programmes, the New Delhi work programme, and the NAPA document structure.'

In [ ]:
var = response["source_documents"]

In [ ]:
len(var)

4

In [415]:
response3 = rag_chain.invoke("Key themes within all the documents, focusing on energy policy, climate adaptation, and mitigation strategies.")
print(response3)

{'query': 'Key themes within all the documents, focusing on energy policy, climate adaptation, and mitigation strategies.', 'result': "Based on the provided documents, I can identify some key themes that are relevant to energy policy, climate adaptation, and mitigation strategies. Here are a few:\n\n1. Energy efficiency and sustainable development: The documents emphasize the importance of integrating adaptation planning with sustainable development, including energy efficiency measures.\n2. Climate change vulnerability and risk assessment: The documents highlight the need for assessing vulnerability and risk associated with climate change, which is essential for developing effective adaptation strategies.\n3. Capacity building and institutional strengthening: The documents stress the importance of capacity building and institutional strengthening to support adaptation efforts, particularly in developing countries.\n4. Integration of climate change into national policies: The documents

In [416]:
response3["result"]

"Based on the provided documents, I can identify some key themes that are relevant to energy policy, climate adaptation, and mitigation strategies. Here are a few:\n\n1. Energy efficiency and sustainable development: The documents emphasize the importance of integrating adaptation planning with sustainable development, including energy efficiency measures.\n2. Climate change vulnerability and risk assessment: The documents highlight the need for assessing vulnerability and risk associated with climate change, which is essential for developing effective adaptation strategies.\n3. Capacity building and institutional strengthening: The documents stress the importance of capacity building and institutional strengthening to support adaptation efforts, particularly in developing countries.\n4. Integration of climate change into national policies: The documents encourage Parties to integrate climate change considerations into their national policies, including energy policy, agriculture, wate

In [417]:
response3["source_documents"]

[Document(metadata={'page': 11, 'source': '/Users/badekale/Documents/Hamoyeew/Subfolders/COP7-2/13a04.pdf'}, page_content='FCCC/CP/2001/13/Add.4\nEnglish\nPage 12\n3.  Identification of key adaptation needs\n13.  Based on this overview and framework, past and current practices for adaptation to\nclimate change and climate variability will be identified as related to existing information\nregarding the country’s vulnerability to the adverse effects of climate change, climate variability\nand extreme weather events, as well as long-term climate change.  This section will explain how\nand to what extent activities may address specific vulnerabilities.\n14.  Given the actual and potential adverse effects of climate change described in section F.2\nabove, this section will identify relevant adaptation options including capacity building, policy\nreform, integration into sectoral policies and project-level activities.\n4.  Criteria for selecting priority activities\n15.  A set of locally-dri